In [209]:
# ===================================================================
#  Library
# ===================================================================
import os
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import mean_absolute_percentage_error

import warnings
warnings.simplefilter("ignore")

from tqdm.auto import tqdm
import optuna
import unicodedata

import lightgbm as lgb

In [210]:
import multiprocessing
NUM_CORES = multiprocessing.cpu_count()
NUM_CORES

8

In [211]:
# ===================================================================
#  CFG
# ===================================================================
class CFG:
    debug = False
    seed = 42
    #seed = 2023
    #seed = 1111
    n_splits = 5
    data_dir = "G:/マイドライブ/signate_StudentCup2023/data/"
    target_bins = 20
    year_bins = 20
    num_boost_round = 500
    stopping_rounds = 50
    n_trials = 100
    save_dir = "G:/マイドライブ/signate_StudentCup2023/exp/"

In [212]:
# ===================================================================
#  Utils
# ===================================================================
def seed_everything(seed):
    """fix random factors"""
    random.seed(seed)
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
seed_everything(CFG.seed)
    

def get_score(y_true, y_pred):
    """get MAPE score"""
    score = mean_absolute_percentage_error(y_true, y_pred)
    return score * 100

In [213]:
# ===================================================================
#  Data Loading
# ===================================================================
train = pd.read_csv(CFG.data_dir+"train.csv")
test = pd.read_csv(CFG.data_dir+"test.csv")

state_coordinate_df = pd.read_csv(CFG.data_dir+"state_coordinate.csv")
region_coordinate_df = pd.read_csv(CFG.data_dir+"region_coordinate.csv")

train["flag"] = "train"
train = pd.merge(train, state_coordinate_df, on="state", how="left")
train = pd.merge(train, region_coordinate_df, on="region", how="left")
test["flag"] = "test"
test = pd.merge(test, state_coordinate_df, on="state", how="left")
test = pd.merge(test, region_coordinate_df, on="region", how="left")
all_data = pd.concat([train, test], ignore_index=True)

In [214]:
# ===================================================================
#  preprocessing
# ===================================================================
# year
def pre_year(df: pd.DataFrame):
    year_dict = {
        2999:1999,
        3008:2008,
        3011:2011,
        3015:2015,
        3017:2017,
        3019:2019,
    }
    df["year"] = df["year"].replace(year_dict)
    return df
all_data = pre_year(all_data)

# manufacturer
all_data["manufacturer"] = all_data["manufacturer"].str.lower().apply(lambda x: unicodedata.normalize('NFKC', x))
manufacturer_map = {
    'niѕsan':'nissan',
    'nisѕan':'nissan',
    'subαru':'subaru',
    'toyotа':'toyota',
    'sαturn':'saturn',
    'аcura':'acura',
    'vоlkswagen':'volkswagen',
    'lexuѕ':'lexus',
    'ᴄhrysler':'chrysler',
}
all_data["manufacturer"] = all_data["manufacturer"].replace(manufacturer_map)

# fuel
## 欠損値 train: 1239, test, 1495 
## わざと欠損値にしている訳ではなさそう？
all_data["fuel"].fillna(value="Unknown", inplace=True)


# odometer
## 後でvalidationを作るときに直そう

# title_status
## 欠損値 train: 456, test: 229
all_data["title_status"].fillna(value="Unknown", inplace=True)

# size
def pre_size(df: pd.DataFrame):
    size_dict = {
        "fullーsize":"full-size",
        "midーsize":"mid-size",
        "subーcompact":"sub-compact",
        "full−size":"full-size",
        "mid−size":"mid-size"
    }
    df["size"] = df["size"].replace(size_dict)
    return df
all_data = pre_size(all_data)

# type
## 欠損値 train: 456, test: 229
all_data["type"].fillna(value="Unknown", inplace=True)


# state
## 欠損値 train: 3304, test: 3029
all_data["state"].fillna(value="Unknown", inplace=True)

In [215]:
# ===================================================================
#  Cross Validation
# ===================================================================
train = all_data[all_data["flag"] == "train"].reset_index(drop=True)
test = all_data[all_data["flag"] == "test"].reset_index(drop=True)

train.sort_values(by="id", ignore_index=True, inplace=True)

train["year_map"], bins = pd.cut(train["year"], bins=20, labels=False, retbins=True)
test["year_map"] = pd.cut(test["year"], bins=bins, labels=False)
train["price_map"] = pd.cut(train["price"], bins=CFG.target_bins, labels=False)

skf = StratifiedKFold(n_splits=CFG.n_splits, shuffle=True, random_state=CFG.seed)
for i, (_, val) in enumerate(skf.split(X=train, y=train["price_map"])):
    train.loc[val, "fold"] = i
print(train["fold"].value_counts())
print(train.groupby("fold")["price"].mean().std())

fold
1.0    5507
0.0    5507
2.0    5506
4.0    5506
3.0    5506
Name: count, dtype: int64
8.784566908616718


In [216]:
# ===================================================================
#  Variance by category
# ===================================================================
all_data = pd.concat([train, test], ignore_index=True)
tmp_ = ['region',  'manufacturer', 'condition', 'cylinders',
       'fuel',  'title_status', 'transmission', 'drive', 'size',
       'type', 'paint_color', 'state',  'year_map',]
df = all_data[(all_data["odometer"] > 100)&(all_data["odometer"] < 400000)].reset_index(drop=True)
tmp = []
for col in tmp_:
    tmp.append(df.groupby(col)["odometer"].mean().std())
df = pd.DataFrame(zip(tmp_, tmp, [len(train[col].unique()) for col in tmp_]), columns=["feature", "std", "len"],)
display(df.sort_values("std", ascending=False, ignore_index=True))

,feature,std,len
0,year_map,33400.067098,20
1,condition,26899.720306,6
2,fuel,19292.666186,6
3,region,17761.052586,372
4,manufacturer,16545.615833,37
5,paint_color,16320.395312,12
6,type,16318.821524,14
7,cylinders,13385.064899,8
8,drive,12651.327741,3
9,title_status,11325.772185,7


In [217]:
# ===================================================================
#  use features
# =================================================================== 
## numerical
CFG.numerical_features = ['year', "odometer"]

## category
CFG.categorical_features = ['drive', "size",'paint_color', "title_status","type", "fuel"]
CFG.target_encoding_features = ['manufacturer','condition', 'transmission',]
CFG.count_encoding_features = ["region",'cylinders',"state"]
CFG.max_encoding_features = ['cylinders']
CFG.std_encoding_features = ["fuel"]
CFG.min_encoding_features = []
CFG.median_encoding_features = ["manufacturer", "condition"]


## use_features
CFG.use_features = list(set(CFG.numerical_features) | set(CFG.categorical_features))

In [218]:
"""
# ===================================================================
#  model
# ===================================================================
preds, oof_df, feature_importance_df = [], pd.DataFrame(), pd.DataFrame()
for fold in range(CFG.n_splits):
    X_train = train[train["fold"] != fold].reset_index(drop=True)
    X_valid = train[train["fold"] == fold].reset_index(drop=True)
    test_df = test.copy()
    
    # odometerの補正
    fillna_map = X_train[(X_train["odometer"] > 100)&(X_train["odometer"] < 400000)].groupby(["year_map"])["odometer"].mean().reset_index()
    def replace_odometer(df: pd.DataFrame, fillna_map: pd.DataFrame):
        df_1 = df[(df["odometer"] < 100)|(df["odometer"] > 400000)].reset_index(drop=True)
        df_2 = df[(df["odometer"] >= 100)&(df["odometer"] <= 400000)].reset_index(drop=True)
        df_1.drop("odometer", inplace=True, axis=1)
        df_1 = pd.merge(df_1, fillna_map, on="year_map", how="left")
        df = pd.concat([df_1, df_2])
        return df.sort_values("id", ignore_index=True)
    X_train = replace_odometer(X_train, fillna_map)
    X_valid = replace_odometer(X_valid, fillna_map)
    test_df = replace_odometer(test_df, fillna_map)
    
    
    def add_odometer_per_year(df: pd.DataFrame):
        df["elapsed_year"] = 2023 - df["year"]
        df["odometer_per_year"] = df["odometer"] / df["elapsed_year"]
        return df
    X_train = add_odometer_per_year(X_train)
    X_valid = add_odometer_per_year(X_valid) 
    test_df = add_odometer_per_year(test_df)
        
    # CFG.target_encoding_features
    for col in CFG.target_encoding_features:
        mean_map = X_train.groupby(col)["price"].mean()
        X_train[col+"_target_encoding"] = X_train[col].map(mean_map)
        X_valid[col+"_target_encoding"] = X_valid[col].map(mean_map)
        test_df[col+"_target_encoding"] = test_df[col].map(mean_map)
        if fold == 0:
            CFG.use_features.append(col+"_target_encoding")
            
        
    # CFG.count_encoding_features
    for col in CFG.count_encoding_features:
        count_map = X_train[col].value_counts().to_dict()
        X_train[col+"_count_encoding"] = X_train[col].map(count_map)
        X_valid[col+"_count_encoding"] = X_valid[col].map(count_map)
        test_df[col+"_count_encoding"] = test_df[col].map(count_map)
        if fold == 0:
            CFG.use_features.append(col+"_count_encoding")
            
    
    # CFG.max_encoding_features
    for col in CFG.max_encoding_features:
        max_map = X_train.groupby(col)["price"].max()
        X_train[col+"_max_encoding"] = X_train[col].map(max_map)
        X_valid[col+"_max_encoding"] = X_valid[col].map(max_map)
        test_df[col+"_max_encoding"] = test_df[col].map(max_map)
        if fold == 0:
            CFG.use_features.append(col+"_max_encoding")
            
            
    # CFG.min_encoding_features
    for col in CFG.min_encoding_features:
        min_map = X_train.groupby(col)["price"].min()
        X_train[col+"_min_encoding"] = X_train[col].map(min_map)
        X_valid[col+"_min_encoding"] = X_valid[col].map(min_map)
        test_df[col+"_min_encoding"] = test_df[col].map(min_map)
        if fold == 0:
            CFG.use_features.append(col+"_min_encoding")
            
            
    # CFG.std_encoding_features
    for col in CFG.std_encoding_features:
        std_map = X_train.groupby(col)["price"].std()
        X_train[col+"_std_encoding"] = X_train[col].map(std_map)
        X_valid[col+"_std_encoding"] = X_valid[col].map(std_map)
        test_df[col+"_std_encoding"] = test_df[col].map(std_map)
        if fold == 0:
            CFG.use_features.append(col+"_std_encoding")
            
    
    # CFG.median_encoding_features
    for col in CFG.median_encoding_features:
        median_map = X_train.groupby(col)["price"].median()
        X_train[col+"_median_encoding"] = X_train[col].map(median_map)
        X_valid[col+"_median_encoding"] = X_valid[col].map(median_map)
        test_df[col+"_median_encoding"] = test_df[col].map(median_map)
        if fold == 0:
            CFG.use_features.append(col+"_median_encoding")
    
    
    # CFG.categorical_featuresをcategory型に変換
    for col in CFG.categorical_features:
        X_train[col] = X_train[col].astype("category")
        X_valid[col] = X_valid[col].astype("category")
        test_df[col] = test_df[col].astype("category")
        
        


    # train
    lgb_train = lgb.Dataset(X_train[CFG.use_features], X_train["price"], categorical_feature = CFG.categorical_features,)
    lgb_valid = lgb.Dataset(X_valid[CFG.use_features], X_valid["price"], categorical_feature = CFG.categorical_features,)


    # params
    lgb_param = {
        "task":"train",
        "objective": "mape",
        "boosting":"gbdt",
        "num_iterations": CFG.num_boost_round,
        "learning_rate":0.1, # default: 0.1
        "num_leaves":31, # max number of leaves in one tree ###
        "max_depth":4, # default -1, int: limit the max depth for tree model ###
        "min_child_weight":1e-3, # double: minimal sum hessian in one leaf
        "min_data_in_leaf":20, # minimal number of data in one leaf
        "alpha":0.9, # double, constraints, alpha > 0.0: 
        "colsample_bytree":1.0, # 0 < "colsample_bytree" < 1
        #: LightGBM will randomly select a subset of features on each iteration (tree) if feature_fraction is smaller than 1.0
        "lambda": 0, #lambda_l2 >= 0.0: L2 regularization
        "subsample":1, #0.0 < bagging_fraction <= 1.0
        "num_threads":1,
        "metric": 'mape',
        "seed" : CFG.seed,
        "verbosity": -1,   
    }
    
    lgbm_params = {
        "objective": "mape",
        "metrics": "mape",
        "n_estimators": 10000,
        "learning_rate": 0.01,
        "max_depth": 5,
        "num_leaves": 32,
        "colsample_bytree": 0.446,
        "subsample": 0.339,
        "importance_type": "gain",
        "random_state": 88,
    }

    model = lgb.train(
                    lgb_param, 
                    lgb_train, 
                    valid_sets=[lgb_valid],
                    categorical_feature = CFG.categorical_features,
                    callbacks=[lgb.early_stopping(stopping_rounds=CFG.stopping_rounds, verbose=False),],
                    )

    # feature importance
    feature_importance_df[f"importance_fold{fold}"] = model.feature_importance(importance_type='gain')

    # eval
    X_valid[f"pred"] = model.predict(X_valid[CFG.use_features], num_iteration=model.best_iteration)
    print(f"fold{fold}:", get_score(y_true=X_valid["price"], y_pred=X_valid["pred"]))

    # concat
    oof_df = pd.concat([oof_df, X_valid], ignore_index=True)

    # pred test
    preds.append(model.predict(test_df[CFG.use_features]))
    
feature_importance_df["feature"] = CFG.use_features
print('\033[32m'+"====== CV score ======"+'\033[0m')
print('\033[32m'+f'{get_score(oof_df["price"], oof_df["pred"])}'+'\033[0m')
predictions = np.mean(preds, axis=0)
"""

'\n# ===================================================================\n#  model\n# ===================================================================\npreds, oof_df, feature_importance_df = [], pd.DataFrame(), pd.DataFrame()\nfor fold in range(CFG.n_splits):\n    X_train = train[train["fold"] != fold].reset_index(drop=True)\n    X_valid = train[train["fold"] == fold].reset_index(drop=True)\n    test_df = test.copy()\n    \n    # odometerの補正\n    fillna_map = X_train[(X_train["odometer"] > 100)&(X_train["odometer"] < 400000)].groupby(["year_map"])["odometer"].mean().reset_index()\n    def replace_odometer(df: pd.DataFrame, fillna_map: pd.DataFrame):\n        df_1 = df[(df["odometer"] < 100)|(df["odometer"] > 400000)].reset_index(drop=True)\n        df_2 = df[(df["odometer"] >= 100)&(df["odometer"] <= 400000)].reset_index(drop=True)\n        df_1.drop("odometer", inplace=True, axis=1)\n        df_1 = pd.merge(df_1, fillna_map, on="year_map", how="left")\n        df = pd.concat([df_1,

| max_depth | CV |
| - | - |
| -1 | 44.61912756990577 | 
| 1 | 46.26899471238079 |
| 2 | 44.72894181750576 |
| 3 | 44.300660325492025 |
| 4 | 44.25284711315097 |
| 5 | 44.3421206027383 |

fold0: 44.81489170450998  
fold1: 45.365720502207544  
fold2: 44.12319504793524  
fold3: 44.57536168304539  
fold4: 44.21629776089228  
====== CV score ======  
44.61912756990577  


In [219]:
for col in CFG.target_encoding_features:
    CFG.use_features.append(col+"_target_encoding")
for col in CFG.count_encoding_features:
    CFG.use_features.append(col+"_count_encoding")
for col in CFG.max_encoding_features:
    CFG.use_features.append(col+"_max_encoding")
for col in CFG.min_encoding_features:
    CFG.use_features.append(col+"_min_encoding")
for col in CFG.std_encoding_features:
    CFG.use_features.append(col+"_std_encoding")
for col in CFG.median_encoding_features:            
    CFG.use_features.append(col+"_median_encoding")

In [220]:
# ===================================================================
#  model
# ===================================================================
if CFG.debug:
    train = train.sample(n=100, ignore_index=True)
    
def objective(trial):
    oof_df =  pd.DataFrame()
    for fold in range(CFG.n_splits):
        X_train = train[train["fold"] != fold].reset_index(drop=True)
        X_valid = train[train["fold"] == fold].reset_index(drop=True)
        
        # odometerの補正
        fillna_map = X_train[(X_train["odometer"] > 100)&(X_train["odometer"] < 400000)].groupby(["year_map"])["odometer"].mean().reset_index()
        def replace_odometer(df: pd.DataFrame, fillna_map: pd.DataFrame):
            df_1 = df[(df["odometer"] < 100)|(df["odometer"] > 400000)].reset_index(drop=True)
            df_2 = df[(df["odometer"] >= 100)&(df["odometer"] <= 400000)].reset_index(drop=True)
            df_1.drop("odometer", inplace=True, axis=1)
            df_1 = pd.merge(df_1, fillna_map, on="year_map", how="left")
            df = pd.concat([df_1, df_2])
            return df.sort_values("id", ignore_index=True)
        X_train = replace_odometer(X_train, fillna_map)
        X_valid = replace_odometer(X_valid, fillna_map)
        
        """
        def add_odometer_per_year(df: pd.DataFrame):
            df["elapsed_year"] = 2023 - df["year"]
            df["odometer_per_year"] = df["odometer"] / df["elapsed_year"]
            return df
        X_train = add_odometer_per_year(X_train)
        X_valid = add_odometer_per_year(X_valid) 
        test_df = add_odometer_per_year(test_df)
        """
            
        # CFG.target_encoding_features
        for col in CFG.target_encoding_features:
            mean_map = X_train.groupby(col)["price"].mean()
            X_train[col+"_target_encoding"] = X_train[col].map(mean_map)
            X_valid[col+"_target_encoding"] = X_valid[col].map(mean_map)
            
        # CFG.count_encoding_features
        for col in CFG.count_encoding_features:
            count_map = X_train[col].value_counts().to_dict()
            X_train[col+"_count_encoding"] = X_train[col].map(count_map)
            X_valid[col+"_count_encoding"] = X_valid[col].map(count_map)
                
        # CFG.max_encoding_features
        for col in CFG.max_encoding_features:
            max_map = X_train.groupby(col)["price"].max()
            X_train[col+"_max_encoding"] = X_train[col].map(max_map)
            X_valid[col+"_max_encoding"] = X_valid[col].map(max_map)
                
        # CFG.min_encoding_features
        for col in CFG.min_encoding_features:
            min_map = X_train.groupby(col)["price"].min()
            X_train[col+"_min_encoding"] = X_train[col].map(min_map)
            X_valid[col+"_min_encoding"] = X_valid[col].map(min_map)
                
        # CFG.std_encoding_features
        for col in CFG.std_encoding_features:
            std_map = X_train.groupby(col)["price"].std()
            X_train[col+"_std_encoding"] = X_train[col].map(std_map)
            X_valid[col+"_std_encoding"] = X_valid[col].map(std_map)
                        
        # CFG.median_encoding_features
        for col in CFG.median_encoding_features:
            median_map = X_train.groupby(col)["price"].median()
            X_train[col+"_median_encoding"] = X_train[col].map(median_map)
            X_valid[col+"_median_encoding"] = X_valid[col].map(median_map)        
        
        # CFG.categorical_featuresをcategory型に変換
        for col in CFG.categorical_features:
            X_train[col] = X_train[col].astype("category")
            X_valid[col] = X_valid[col].astype("category")

        # train
        lgb_train = lgb.Dataset(X_train[CFG.use_features], X_train["price"], categorical_feature = CFG.categorical_features,)
        lgb_valid = lgb.Dataset(X_valid[CFG.use_features], X_valid["price"], categorical_feature = CFG.categorical_features,)

        lgb_param = {
            "task":"train",
            "objective": "mape",
            "boosting":"gbdt",
            "num_iterations": CFG.num_boost_round,
            "learning_rate": 0.1, # default: 0.1
            "min_child_weight":1e-3, # double: minimal sum hessian in one leaf
            "subsample":1, #0.0 < bagging_fraction <= 1.0
            "num_threads":NUM_CORES,
            "metric": 'mape',
            "seed" : CFG.seed,
            "verbosity": -1,   
            "max_depth": trial.suggest_int("max_depth", 1, 10),
            "num_leaves": trial.suggest_int("num_leaves", 2, 100), # < 2**"max_depth"
            "alpha": trial.suggest_float("alpha", 1e-6, 10),
            "lambda": trial.suggest_float("lambda", 1e-6, 10),
            "colsample_bytree":trial.suggest_float("colsample_bytree", 0, 1),
            "min_child_samples": trial.suggest_int("min_child_samples", 5, 100),
            "feature_fraction": trial.suggest_float("feature_fraction", 0.4, 1.0),
            "bagging_fraction": trial.suggest_float("bagging_fraction", 0.4, 1.0),
            "bagging_freq": trial.suggest_int("bagging_freq", 1, 7),
        }

        model = lgb.train(
                        lgb_param, 
                        lgb_train, 
                        valid_sets=[lgb_valid],
                        categorical_feature = CFG.categorical_features,
                        callbacks=[lgb.early_stopping(stopping_rounds=CFG.stopping_rounds, verbose=False),],
                        )
        # eval
        X_valid[f"pred"] = model.predict(X_valid[CFG.use_features], num_iteration=model.best_iteration)
        # concat
        oof_df = pd.concat([oof_df, X_valid], ignore_index=True)
    return get_score(oof_df["price"], oof_df["pred"])
    
optuna.logging.set_verbosity(optuna.logging.WARNING)
study = optuna.create_study(direction="minimize",
                            sampler=optuna.samplers.TPESampler(seed=CFG.seed))
study.optimize(objective, 
                n_trials=CFG.n_trials,
                show_progress_bar=True)

best_params = study.best_params
best_value = study.best_value
print(best_params, best_value)

  0%|          | 0/100 [00:00<?, ?it/s]

Best trial: 72. Best value: 44.2089: 100%|██████████| 100/100 [20:55<00:00, 12.55s/it]

{'max_depth': 5, 'num_leaves': 97, 'alpha': 3.0259790706186402, 'lambda': 7.646516124349529, 'colsample_bytree': 0.22199279683162915, 'min_child_samples': 81, 'feature_fraction': 0.4670722972795847, 'bagging_fraction': 0.98728869364854, 'bagging_freq': 3} 44.20890187548654


In [229]:
from optuna.visualization import plot_parallel_coordinate
from optuna.visualization import plot_param_importances

In [230]:
plot_parallel_coordinate(study)

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'dimensions': [{'label': 'Objective Value',
                              'range': [44.20890187548654, 46.367573863616656],
                              'values': [44.26613191730135, 45.02595587682659,
                                         44.3507389885141, 44.38430020649219,
                                         44.418205242699, 44.327342550037756,
                                         44.442141457015346, 44.32869307991435,
                                         46.367573863616656, 44.40978282279831,
                                         44.47992054012187, 44.61304079800378,
                                         44.36020059537444, 44.373764528891904,
                                         44.5395356233537, 44.314663049064286,
                                         44.35348476325762, 44.36271920065812,
                                         44.55213525788126, 46.300811275935985,
                                         44.351285682643024, 44.435519914370616,
                                         44.265832533382344, 44.339290896284766,
                                         44.88094302258532, 44.50467734916339,
                                         44.64835603322172, 44.382850517492415,
                                         44.384499126099364, 44.49817640049199,
                                         44.399062855017576, 44.373601970793494,
                                         44.50824253106697, 44.308260756203566,
                                         44.36919569080932, 44.51017091086822,
                                         44.33667785196861, 44.428516170458124,
                                         44.33580766594878, 44.29610159158736,
                                         44.305054775659755, 44.29244779011677,
                                         44.32581835637505, 44.27736948785386,
                                         44.425281993826786, 44.30550661113876,
                                         44.38370702918657, 44.37172260200653,
                                         44.264447318962304, 44.30355219670758,
                                         44.366254610670225, 44.378070373083276,
                                         44.31774320028066, 44.2566233586265,
                                         44.230726389720026, 44.31598094137389,
                                         44.24205464287048, 44.35209651162046,
                                         44.24495402041233, 44.26200509604406,
                                         44.59993498878382, 44.349131753798396,
                                         44.29349310178346, 44.26997983168163,
                                         44.41449467649419, 44.28730213561426,
                                         44.25107407134262, 44.34539163497779,
                                         44.3075681882151, 44.28663551979785,
                                         44.31969448860214, 44.30523704097535,
                                         44.20890187548654, 44.29855465355128,
                                         44.29168343763577, 44.29371218925505,
                                         44.292049304641985, 44.25195629470676,
                                         44.38570193041323, 44.29648516462272,
                                         44.24275242779013, 44.26204026890642,
                                         44.28217759512446, 44.23107120959496,
                                         44.24315414534964, 44.2209761407907,
                                         44.223314420925774, 44.29378197708056,
                                         44.28709533155107, 44.27777448977976,
                                         44.26597361752531, 44.248030217267065,
                                         44.24402536673639, 44.274638127775056,
                                         44.30857733484811, 44.30201775381615,
                                         

In [231]:
plot_param_importances(study)

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'cliponaxis': False,
              'hovertemplate': [lambda (FloatDistribution):
                                0.0013963320571787712<extra></extra>,
                                feature_fraction (FloatDistribution):
                                0.003124326665495751<extra></extra>,
                                min_child_samples (IntDistribution):
                                0.008426470469576966<extra></extra>, alpha
                                (FloatDistribution):
                                0.030645985473193027<extra></extra>, bagging_freq
                                (IntDistribution):
                                0.060808640625163723<extra></extra>, num_leaves
                                (IntDistribution):
                                0.06632607874069955<extra></extra>,
                                bagging_fraction (FloatDistribution):
                                0.09297402763217748<extra></extra>,
                                colsample_bytree (FloatDistribution):
                                0.1785646595657753<extra></extra>, max_depth
                                (IntDistribution):
                                0.5577334787707392<extra></extra>],
              'marker': {'color': 'rgb(66,146,198)'},
              'orientation': 'h',
              'text': [<0.01, <0.01, <0.01, 0.03, 0.06, 0.07, 0.09, 0.18, 0.56],
              'textposition': 'outside',
              'type': 'bar',
              'x': [0.0013963320571787712, 0.003124326665495751,
                    0.008426470469576966, 0.030645985473193027,
                    0.060808640625163723, 0.06632607874069955, 0.09297402763217748,
                    0.1785646595657753, 0.5577334787707392],
              'y': [lambda, feature_fraction, min_child_samples, alpha,
                    bagging_freq, num_leaves, bagging_fraction, colsample_bytree,
                    max_depth]}],
    'layout': {'showlegend': False,
               'template': '...',
               'title': {'text': 'Hyperparameter Importances'},
               'xaxis': {'title': {'text': 'Importance for Objective Value'}},
               'yaxis': {'title': {'text': 'Hyperparameter'}}}
})

In [232]:
print(best_params, best_value)

{'max_depth': 5, 'num_leaves': 97, 'alpha': 3.0259790706186402, 'lambda': 7.646516124349529, 'colsample_bytree': 0.22199279683162915, 'min_child_samples': 81, 'feature_fraction': 0.4670722972795847, 'bagging_fraction': 0.98728869364854, 'bagging_freq': 3} 44.20890187548654


In [222]:
# ===================================================================
#  plot feature importance
# ===================================================================
"""
feature_importance_df.fillna(value=0, inplace=True)
feature_importance_df["average"] = feature_importance_df.loc[:, 'importance_fold0':f'importance_fold{CFG.n_splits-1}'].mean(axis=1)
best_features = feature_importance_df[["feature", "average"]].groupby("feature", as_index=False).mean().sort_values(by="average", ascending=False).head(50)
plt.figure(figsize=(8, 16))
sns.barplot(x="average", y="feature", data=best_features)
plt.title('Features importance (averaged/folds)')
plt.tight_layout()
"""

'\nfeature_importance_df.fillna(value=0, inplace=True)\nfeature_importance_df["average"] = feature_importance_df.loc[:, \'importance_fold0\':f\'importance_fold{CFG.n_splits-1}\'].mean(axis=1)\nbest_features = feature_importance_df[["feature", "average"]].groupby("feature", as_index=False).mean().sort_values(by="average", ascending=False).head(50)\nplt.figure(figsize=(8, 16))\nsns.barplot(x="average", y="feature", data=best_features)\nplt.title(\'Features importance (averaged/folds)\')\nplt.tight_layout()\n'

In [223]:
# ===================================================================
#  optuna
# ===================================================================
"""
def objective(trial):
    a = trial.suggest_float("a", 0, 1)
    
    oof_df[f"pred_1"] = oof_df[f"pred"] * a
    score = get_score(y_true=oof_df["price"], y_pred = oof_df[f"pred_1"])
    return score
    
optuna.logging.set_verbosity(optuna.logging.WARNING)
study = optuna.create_study(direction="minimize",
                            sampler=optuna.samplers.TPESampler(seed=CFG.seed))
study.optimize(objective, 
                n_trials=CFG.n_trials,
                show_progress_bar=True)

best_params = study.best_params
best_value = study.best_value
print(best_params, best_value)
"""

'\ndef objective(trial):\n    a = trial.suggest_float("a", 0, 1)\n    \n    oof_df[f"pred_1"] = oof_df[f"pred"] * a\n    score = get_score(y_true=oof_df["price"], y_pred = oof_df[f"pred_1"])\n    return score\n    \noptuna.logging.set_verbosity(optuna.logging.WARNING)\nstudy = optuna.create_study(direction="minimize",\n                            sampler=optuna.samplers.TPESampler(seed=CFG.seed))\nstudy.optimize(objective, \n                n_trials=CFG.n_trials,\n                show_progress_bar=True)\n\nbest_params = study.best_params\nbest_value = study.best_value\nprint(best_params, best_value)\n'

In [224]:
"""
# ===================================================================
#  prediction
# ===================================================================
test["pred"] = predictions * best_params["a"]
print(test["pred"].min(), test["pred"].max())
test[["id", "pred"]].to_csv(CFG.save_dir+"exp004.csv", index=False, header=None)
"""

'\n# ===================================================================\n#  prediction\n# ===================================================================\ntest["pred"] = predictions * best_params["a"]\nprint(test["pred"].min(), test["pred"].max())\ntest[["id", "pred"]].to_csv(CFG.save_dir+"exp004.csv", index=False, header=None)\n'